In [1]:
# packages

import numpy as np
import math as m
import pandas as pd
import matplotlib.pyplot as plt
import scipy.integrate as integrate
from scipy.optimize import fsolve
from sympy import symbols, Eq, solve, I, cos, exp, pi
import os
os.chdir('C:\\Users\\nicol\\Desktop\\pythonfiles\\Aeroelasticity\\assignment1')  # change this to your directory


In [2]:
# functions

def compute_matrices(theta_blade, theta_yaw, theta_tilt, theta_cone):

    a1 = (  [1,0,0],
            [0, np.cos(theta_yaw), np.sin(theta_yaw)],
            [0, -np.sin(theta_yaw), np.cos(theta_yaw)])
    a2 = (  [np.cos(theta_tilt), 0, -np.sin(theta_tilt)],
            [0, 1, 0],
            [np.sin(theta_tilt), 0, np.cos(theta_tilt)])
    a12 = np.dot(a1, a2)
    a21 = np.transpose(a12)
    
    a23 = ( [np.cos(theta_blade), np.sin(theta_blade), 0],
            [-np.sin(theta_blade), np.cos(theta_blade), 0],
            [0, 0, 1])
    a34 = ( [np.cos(theta_cone), 0, -np.sin(theta_cone)],
            [0, 1, 0],
            [np.sin(theta_cone), 0, np.cos(theta_cone)])
    a14 = np.dot(np.dot(a34,a23), a12)
    a41 = np.transpose(a14)

    return a21, a23, a41 

def compute_position(a21, a41, H, Ls, radius):
        
    rt = [H,0,0]
    rs = np.dot(a21, [0,0,-Ls])
    rb = np.dot(a41, [radius,0,0])
    position = rt + rs + rb
    return position

def compute_tower_radius(x):
    if x <= H:
        a = a0
    else:
        a = 0
    return a


In [7]:
# interpolate airfoil

files=[ 'FFA-W3-241.txt',
        'FFA-W3-301.txt',
        'FFA-W3-360.txt',
        'FFA-W3-480.txt',
        'FFA-W3-600.txt',
        'cylinder.txt']

#Initializing tables    
cl_tab=np.zeros([105,6])
cd_tab=np.zeros([105,6])
cm_tab=np.zeros([105,6])
aoa_tab=np.zeros([105,])

#Reading of tables. Only do this once at startup of simulation
for i in range(np.size(files)):
    aoa_tab[:],cl_tab[:,i],cd_tab[:,i],cm_tab[:,i] = np.loadtxt(files[i], skiprows=0).T

# Thickness of the airfoils considered
# NOTE THAT IN PYTHON THE INTERPOLATION REQUIRES THAT THE VALUES INCREASE IN THE VECTOR!

thick_prof=np.zeros(6)
thick_prof[0]=24.1
thick_prof[1]=30.1
thick_prof[2]=36
thick_prof[3]=48
thick_prof[4]=60
thick_prof[5]=100

def force_coeffs_10MW(angle_of_attack,thick,aoa_tab,cl_tab,cd_tab,cm_tab):
    
    cl_aoa=np.zeros([1,6])
    cd_aoa=np.zeros([1,6])
    cm_aoa=np.zeros([1,6])
    
    #Interpolate to current angle of attack:
    for i in range(np.size(files)):
        cl_aoa[0,i]=np.interp (angle_of_attack,aoa_tab,cl_tab[:,i])
        cd_aoa[0,i]=np.interp (angle_of_attack,aoa_tab,cd_tab[:,i])
        cm_aoa[0,i]=np.interp (angle_of_attack,aoa_tab,cm_tab[:,i])
    
    #Interpolate to current thickness:
    cl=np.interp (thick,thick_prof,cl_aoa[0,:])
    cd=np.interp (thick,thick_prof,cd_aoa[0,:])
    cm=np.interp (thick,thick_prof,cm_aoa[0,:])
    
    return cl, cd, cm 

In [5]:
# data

ac=1/3
eps=1e-8
max_iteration=100000
B=3
rho=1.225
R=89.17

blade=pd.read_csv('bladedat.txt',delimiter='\t',header=None,names=['r','beta','c','t_perc'])
r_array = blade['r']



In [3]:
# BEM

#H = 119
#Ls = 7.1
#R = 89.15
#theta_tilt = np.deg2rad(0)
#nu = 0.2
#V0 = 10
#
#num =80
#deltat = 0.15
#
#omega = 0.62
#radius = 70
#
#a0 = 3.32
#
#theta_yaw_a = 0
#theta_yaw_b = np.deg2rad(20)

# angles
theta_yaw = 0
theta_tilt = 0
theta_cone = 0


def BEM():
    #time_array = []
    #theta_blade1_array = []
    #V0_1array = []

    theta_blade1_old = 0

    for n in range(0,num):

        # time
        time = n*deltat
        #time_array.append(time)

        # angles
        theta_blade1 = theta_blade1_old + omega*deltat
        #theta_blade2 = theta_blade1 + 2*np.pi/3
        #theta_blade3 = theta_blade2 + 4*np.pi/3
        #theta_blade1_array.append(theta_blade1_new)

        # running only one blade

        a21_1, a23_1, a41_1 = compute_matrices(theta_blade1_new, theta_yaw, theta_tilt)

        # position
        for i in range(len(r_array[:-1])):
        
            r = blade['r'][i]
            beta = np.deg2rad(blade['beta'][i])
            c = blade['c'][i]
        
        t_perc = blade['t_perc'][i]
        r_1 = compute_position(a21_1, a41_1, H, Ls, radius)

        # shear
        V0x = V0*(r_1[0] / H)**nu

        # tower
        y = r_1[1]
        z = r_1[2]
        r = np.sqrt(y**2 + z**2)
        a = compute_tower_radius(r_1[0])

        Vr =     z/r * V0x * (1 - (a/r)**2)
        Vtheta = y/r * V0x * (1 + (a/r)**2)

        Vx = 0
        Vz =  z/r * Vr + y/r * Vtheta
        Vy =  y/r * Vr - z/r * Vtheta

        # total velocity
        a14_1 = np.transpose(a41_1)
        V0_1_1 = [Vx, Vy, Vz]
        V0_4_1 = np.dot(a14_1, V0_1_1)  
        V0_1array.append(V0_4_1)

        theta_blade1 = theta_blade1_new
    
    V0x_1 = [arr[0] for arr in V0_1array]
    V0y_1 = [arr[1] for arr in V0_1array]
    V0z_1 = [arr[2] for arr in V0_1array]

    return V0x_1, V0y_1, V0z_1, time_array, theta_blade1_array

V0x_1_a, V0y_1_a, V0z_1_a, time_array, theta_blade1_array = compute_velocity_shear_tower(num=num, theta_yaw=theta_yaw_a)
V0x_1_b, V0y_1_b, V0z_1_b, time_array, theta_blade1_array = compute_velocity_shear_tower(num=num, theta_yaw=theta_yaw_b)


plt.figure(1)
plt.plot(np.rad2deg(theta_blade1_array), V0x_1_a, color='r', label=np.rad2deg(theta_yaw_a))
plt.plot(np.rad2deg(theta_blade1_array), V0x_1_b, color='b', label=np.rad2deg(theta_yaw_b))
plt.legend()
plt.xlabel('theta')
plt.ylabel('V0x')
plt.title('blade 1 - x velocity')

plt.figure(2)
plt.plot(np.rad2deg(theta_blade1_array), V0y_1_a, color='r', label=np.rad2deg(theta_yaw_a))
plt.plot(np.rad2deg(theta_blade1_array), V0y_1_b, color='b', label=np.rad2deg(theta_yaw_b))
plt.legend()
plt.xlabel('theta')
plt.ylabel('V0y')
plt.title('blade 1 - y velocity')

plt.figure(3)
plt.plot(np.rad2deg(theta_blade1_array), V0z_1_a, color='r', label=np.rad2deg(theta_yaw_a))
plt.plot(np.rad2deg(theta_blade1_array), V0z_1_b, color='b', label=np.rad2deg(theta_yaw_b))
plt.legend()
plt.xlabel('theta')
plt.ylabel('V0z')
plt.title('blade 1 - z velocity')


NameError: name 'theta_cone' is not defined